Goal: Create a program to study the consequences of establishing habitat corridors between habitat patches

Your framework will have three classes: Individual, Population, and Landscape. Individuals exist in populations and populations exist in landscapes.

In every time step, individuals can stay in the population where they started or move to a new population, the probability of which is stored in a dispersal matrix.

Individuals need to have: phenotype attribute

Populations need: list of individuals
    Constructor to create new list for ind in pop
    Methods to add and remove ind
    Calc and print the freq of phenotypes among ind
    
Lanscapes need: list of populations, dispersal matrix
    Constructor to create new list for populations in that landscape
    Method to use probabilities in dispersal matrix to determine if ind moves or stays each time

Dispersal matrix - consists of the probabilities that an ind moves populations or stays in place each time step. 
    For this species (Warbling Babblers) this corresponds to the possibility of moving in a week
    If an ind moves pops, remove it from the list of ind in the pop it started in and add it to the list in the pop it goes to
        Do not modify a list that you are looping through inside of the loop itself, do it once the loop is complete
    
Notes:
    Population or landscape constructors should set the starting population sizes
    Total number of ind across all populations should stay constant, but ind pops change sizes
    
When simulation begins, all ind in a population should have the same phenotype. As ind move between pop through time steps (weeks), track the trajectory of phenotype freq and pop sizes
    Will need to write a function to create these plots, either as a method of one of the classes or independent 
    Can overlay pt freq or pop sizes for diff pop on a single plt for easy comparison 

In [ ]:
#Import any needed modules
import random

In [ ]:
#Create individual class, with each individual having a phenotype attribute
class individual:
    """Class to hold information about individuals"""
    
    #Create constructor to construct individuals
    def __init__(self,id,pt)
    """Used to create individuals and assign them an id and phenotype(pt)"""
    
    self.id = id
    self.pt = pt




#Create population class
class population:
    """Class to hold information about populations"""
    
    #Create constructor to create lists of individuals in the population
    def __init__(self,id,pt,popSize):
        """Creates list of individuals in the population"""
        
        self.popSize = popSize
        
        #Create list of individuals in population
        self.popIndividuals = []
        
        #Create individuals and add them to the population list
        for num in range(popSize):
            self.popIndividuals.append(individal(id=num))
    
    #Method to add and remove individuals from the populations
    def movePops(self):
        """Randomly adds and removes individuals to and from populations"""
        
    #Method to calculate and print the frequency of phenotypes in the population
    def calcPtFreq(self):
        """Calculates the frequency of phenotypes in the population"""
        
        
        print("The frequency of phenotype %s in this population is %d" % (pt,[frequency]))





#Create landscape class
class landscape:
    """Class to hold information about landscapes"""
    
    #Create constructor to create list of populations in that landscape, and assign each population a number
    def __init__(self,popid,popnum=2)
    
    self.popid = popid
    self.popnum = popnum
    
    #create list of populations in that landscape
    self.popList = []
    
    #Create populations and add them to the list
    for num in range(popnum):
        self.poplist.append(population%d(popid=num) % (num))
        
    #with methods to use the probabilities in a dispersal matrix to determine if the individual moves or stays
    def probMove(self):
        """This method contains a dispersal matrix to determine the probability of if an individual moves or stays in a population"""